In [2]:
import numpy as np

def extract_features_from_mediapipe(hand_landmarks, handedness_label):
    """
    Превращает MediaPipe hand_landmarks и handedness_label
    в вектор признаков того же формата, что использовался для обучения:
    x0,y0,z0,...,x20,y20,z20,is_right_hand

    Args:
        hand_landmarks: один объект hand_landmarks из MediaPipe.
        handedness_label: "Right" или "Left"

    Returns:
        np.array shape (1,64) готовый для scaler/model
    """
    is_right = 1 if handedness_label == "Right" else 0

    # Wrist для нормализации
    wrist = hand_landmarks.landmark[0]
    wx, wy, wz = wrist.x, wrist.y, wrist.z

    # Масштаб — максимальная дистанция от запястья
    distances = [np.linalg.norm([lm.x - wx, lm.y - wy, lm.z - wz])
                 for lm in hand_landmarks.landmark]
    scale = max(distances) if max(distances) > 1e-6 else 1e-6

    coords = []
    for lm in hand_landmarks.landmark:
        coords.extend([
            (lm.x - wx) / scale,
            (lm.y - wy) / scale,
            (lm.z - wz) / scale
        ])

    coords.append(is_right)

    return np.array(coords, dtype=np.float32).reshape(1, -1)


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import time
import copy

def capture_hands_to_csv(
    output_csv,
    gesture_label,
    max_samples=100,
    capture_interval=0.1,
    frame_width=640,
    frame_height=480,
):
    """
    Сбор данных жестов рук.
    - Координаты совпадают с MediaPipe (не зеркальные).
    - Левую и правую руку MediaPipe определяет корректно.
    - Пользователь видит изображение без зеркального эффекта.
    """

    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils

    hands = mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=1,
        min_detection_confidence=0.9,
        min_tracking_confidence=0.7
    )

    rows = []
    count = 0
    last_capture_time = 0

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

    if not cap.isOpened():
        print("!!! Unable to access camera")
        return

    print(f"Camera ready. Capturing every {capture_interval}s. Press 'q' to quit.")

    while cap.isOpened() and count < max_samples:
        success, frame = cap.read()
        if not success:
            print("!!! Can't read frame")
            break

        success, frame = cap.read()
        if not success:
            break

            # Берём кадр с камеры
        success, frame = cap.read()
        if not success:
            break

        # Зеркалим горизонтально (flipCode=1)
        frame_mirrored = cv2.flip(frame, 1)

        # Передаём зеркальный кадр в MediaPipe
        image_rgb = cv2.cvtColor(frame_mirrored, cv2.COLOR_BGR2RGB)
        result = hands.process(image_rgb)


        # --- Кадр для отображения пользователю (зеркальное отображение) ---
        frame_flipped = cv2.flip(frame, 1)

        # Рисуем landmarks на зеркальном кадре для пользователя
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp.solutions.drawing_utils.draw_landmarks(
                    frame_flipped, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )

        cv2.imshow("Hand Capture", frame_flipped)


        label = "None"
        if result.multi_hand_landmarks and result.multi_handedness:
            hand_landmarks = result.multi_hand_landmarks[0]
            label = result.multi_handedness[0].classification[0].label
            is_right = 1 if label == "Right" else 0

            coords = []
            wrist_x = hand_landmarks.landmark[0].x
            wrist_y = hand_landmarks.landmark[0].y
            wrist_z = hand_landmarks.landmark[0].z

            # Масштаб руки для нормализации
            distances = [np.linalg.norm([
                lm.x - wrist_x,
                lm.y - wrist_y,
                lm.z - wrist_z
            ]) for lm in hand_landmarks.landmark]
            scale = max(distances)

            for lm in hand_landmarks.landmark:
                x, y, z = lm.x, lm.y, lm.z
                x = (x - wrist_x) / scale
                y = (y - wrist_y) / scale
                z = (z - wrist_z) / scale
                coords.extend([x, y, z])

            rows.append(coords + [gesture_label, is_right])
            count += 1
            last_capture_time = time.time()
            print(f"✅ Sample {count}/{max_samples} captured ({label} hand, gesture: {gesture_label})")

            # Рисуем руку для пользователя
            # --- Создаем зеркальные landmarks для отображения ---
            # flipped_landmarks = copy.deepcopy(hand_landmarks)
            # for lm in flipped_landmarks.landmark:
            #     lm.x = 1 - lm.x
            # mp_drawing.draw_landmarks(frame_flipped, flipped_landmarks, mp_hands.HAND_CONNECTIONS)


        # Показываем изображение с корректной ориентацией
        text = f"{label} hand | Gesture: {gesture_label}"
        cv2.putText(frame_flipped, text, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow("Hand Capture", frame_flipped)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

        # Ждём до следующего снимка
        while time.time() - last_capture_time < capture_interval:
            time.sleep(0.01)

    cap.release()
    cv2.destroyAllWindows()
    hands.close()

    if not rows:
        print("No samples recorded.")
        return

    # Сохраняем CSV
    columns = [f"{axis}{i}" for i in range(21) for axis in ("x", "y", "z")] + ["gesture", "is_right_hand"]
    df = pd.DataFrame(rows, columns=columns)
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    if os.path.exists(output_csv):
        df_existing = pd.read_csv(output_csv)
        df = pd.concat([df_existing, df], ignore_index=True)

    df.to_csv(output_csv, index=False)
    print(f"\nCSV saved: {output_csv} ({len(df)} total rows)")


In [4]:
gesture_label="question" 
output_csv = '../data/raw/upd_question.csv'

capture_hands_to_csv(
        output_csv=output_csv,
        gesture_label=gesture_label,
        max_samples=500,
        capture_interval=0.2
    )

I0000 00:00:1763296535.449151   95639 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1763296535.459221   95713 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.0-34-generic)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1763296535.514218   95689 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763296535.540836   95688 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Camera ready. Capturing every 0.2s. Press 'q' to quit.


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/fateee/Desktop/ulyana/Hand-Gesture-Recognition/.venv/lib/python3.12/site-packages/cv2/qt/plugins"
W0000 00:00:1763296536.067726   95688 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


✅ Sample 1/500 captured (Right hand, gesture: question)
✅ Sample 2/500 captured (Right hand, gesture: question)
✅ Sample 3/500 captured (Right hand, gesture: question)
✅ Sample 4/500 captured (Right hand, gesture: question)
✅ Sample 5/500 captured (Right hand, gesture: question)
✅ Sample 6/500 captured (Right hand, gesture: question)
✅ Sample 7/500 captured (Right hand, gesture: question)
✅ Sample 8/500 captured (Right hand, gesture: question)
✅ Sample 9/500 captured (Right hand, gesture: question)
✅ Sample 10/500 captured (Right hand, gesture: question)
✅ Sample 11/500 captured (Right hand, gesture: question)
✅ Sample 12/500 captured (Right hand, gesture: question)
✅ Sample 13/500 captured (Right hand, gesture: question)
✅ Sample 14/500 captured (Right hand, gesture: question)
✅ Sample 15/500 captured (Right hand, gesture: question)
✅ Sample 16/500 captured (Right hand, gesture: question)
✅ Sample 17/500 captured (Right hand, gesture: question)
✅ Sample 18/500 captured (Right hand, ge

KeyboardInterrupt: 

In [ ]:
gesture_label="if" 
output_csv = '../data/raw/upd_if.csv'

capture_hands_to_csv(
        output_csv=output_csv,
        gesture_label=gesture_label,
        max_samples=500,
        capture_interval=0.2
    )

I0000 00:00:1763296159.446309   64322 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1763296159.449842   95284 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.0-34-generic)
W0000 00:00:1763296159.474132   95266 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763296159.485162   95278 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Camera ready. Capturing every 0.2s. Press 'q' to quit.


NameError: name 'lm' is not defined

: 

In [40]:
gesture_label="civilian" 
output_csv = '../data/raw/upd_civilian.csv'

capture_hands_to_csv(
        output_csv=output_csv,
        gesture_label=gesture_label,
        max_samples=500,
        capture_interval=0.2
    )

I0000 00:00:1763285486.554687   64322 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1763285486.558430   84668 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.0-34-generic)
W0000 00:00:1763285486.580510   84653 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763285486.596319   84662 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Camera ready. Capturing every 0.2s. Press 'q' to quit.
✅ Sample 1/500 captured (Left hand, gesture: civilian)
✅ Sample 2/500 captured (Left hand, gesture: civilian)
✅ Sample 3/500 captured (Left hand, gesture: civilian)
✅ Sample 4/500 captured (Left hand, gesture: civilian)
✅ Sample 5/500 captured (Left hand, gesture: civilian)
✅ Sample 6/500 captured (Left hand, gesture: civilian)
✅ Sample 7/500 captured (Left hand, gesture: civilian)
✅ Sample 8/500 captured (Left hand, gesture: civilian)
✅ Sample 9/500 captured (Left hand, gesture: civilian)
✅ Sample 10/500 captured (Left hand, gesture: civilian)
✅ Sample 11/500 captured (Left hand, gesture: civilian)
✅ Sample 12/500 captured (Left hand, gesture: civilian)
✅ Sample 13/500 captured (Left hand, gesture: civilian)
✅ Sample 14/500 captured (Left hand, gesture: civilian)
✅ Sample 15/500 captured (Left hand, gesture: civilian)
✅ Sample 16/500 captured (Left hand, gesture: civilian)
✅ Sample 17/500 captured (Left hand, gesture: civilian)
✅ 

In [42]:
gesture_label="cool" 
output_csv = '../data/raw/upd_cool.csv'

capture_hands_to_csv(
        output_csv=output_csv,
        gesture_label=gesture_label,
        max_samples=500,
        capture_interval=0.15
    )

I0000 00:00:1763285774.655583   64322 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1763285774.660025   84846 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.0-34-generic)
W0000 00:00:1763285774.681077   84828 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763285774.698382   84840 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Camera ready. Capturing every 0.15s. Press 'q' to quit.
✅ Sample 1/500 captured (Right hand, gesture: cool)
✅ Sample 2/500 captured (Right hand, gesture: cool)
✅ Sample 3/500 captured (Right hand, gesture: cool)
✅ Sample 4/500 captured (Right hand, gesture: cool)
✅ Sample 5/500 captured (Right hand, gesture: cool)
✅ Sample 6/500 captured (Right hand, gesture: cool)
✅ Sample 7/500 captured (Right hand, gesture: cool)
✅ Sample 8/500 captured (Right hand, gesture: cool)
✅ Sample 9/500 captured (Right hand, gesture: cool)
✅ Sample 10/500 captured (Right hand, gesture: cool)
✅ Sample 11/500 captured (Right hand, gesture: cool)
✅ Sample 12/500 captured (Right hand, gesture: cool)
✅ Sample 13/500 captured (Right hand, gesture: cool)
✅ Sample 14/500 captured (Right hand, gesture: cool)
✅ Sample 15/500 captured (Right hand, gesture: cool)
✅ Sample 16/500 captured (Right hand, gesture: cool)
✅ Sample 17/500 captured (Right hand, gesture: cool)
✅ Sample 18/500 captured (Right hand, gesture: cool)

In [44]:
gesture_label="don" 
output_csv = '../data/raw/upd_don.csv'

capture_hands_to_csv(
        output_csv=output_csv,
        gesture_label=gesture_label,
        max_samples=500,
        capture_interval=0.2
    )

I0000 00:00:1763286068.125859   64322 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1763286068.130189   85054 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.0-34-generic)
W0000 00:00:1763286068.152893   85039 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763286068.167017   85045 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Camera ready. Capturing every 0.2s. Press 'q' to quit.
✅ Sample 1/500 captured (Right hand, gesture: don)
✅ Sample 2/500 captured (Right hand, gesture: don)
✅ Sample 3/500 captured (Right hand, gesture: don)
✅ Sample 4/500 captured (Right hand, gesture: don)
✅ Sample 5/500 captured (Right hand, gesture: don)
✅ Sample 6/500 captured (Right hand, gesture: don)
✅ Sample 7/500 captured (Right hand, gesture: don)
✅ Sample 8/500 captured (Right hand, gesture: don)
✅ Sample 9/500 captured (Right hand, gesture: don)
✅ Sample 10/500 captured (Right hand, gesture: don)
✅ Sample 11/500 captured (Right hand, gesture: don)
✅ Sample 12/500 captured (Right hand, gesture: don)
✅ Sample 13/500 captured (Right hand, gesture: don)
✅ Sample 14/500 captured (Right hand, gesture: don)
✅ Sample 15/500 captured (Right hand, gesture: don)
✅ Sample 16/500 captured (Right hand, gesture: don)
✅ Sample 17/500 captured (Right hand, gesture: don)
✅ Sample 18/500 captured (Right hand, gesture: don)
✅ Sample 19/500 ca

In [46]:
gesture_label="mafia" 
output_csv = '../data/raw/upd_mafia.csv'

capture_hands_to_csv(
        output_csv=output_csv,
        gesture_label=gesture_label,
        max_samples=500,
        capture_interval=0.1
    )

I0000 00:00:1763286362.078567   64322 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1763286362.082587   85222 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.1), renderer: AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.0-34-generic)
W0000 00:00:1763286362.104045   85204 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763286362.121640   85203 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Camera ready. Capturing every 0.1s. Press 'q' to quit.
✅ Sample 1/500 captured (Right hand, gesture: mafia)
✅ Sample 2/500 captured (Right hand, gesture: mafia)
✅ Sample 3/500 captured (Right hand, gesture: mafia)
✅ Sample 4/500 captured (Right hand, gesture: mafia)
✅ Sample 5/500 captured (Right hand, gesture: mafia)
✅ Sample 6/500 captured (Right hand, gesture: mafia)
✅ Sample 7/500 captured (Right hand, gesture: mafia)
✅ Sample 8/500 captured (Right hand, gesture: mafia)
✅ Sample 9/500 captured (Right hand, gesture: mafia)
✅ Sample 10/500 captured (Right hand, gesture: mafia)
✅ Sample 11/500 captured (Right hand, gesture: mafia)
✅ Sample 12/500 captured (Right hand, gesture: mafia)
✅ Sample 13/500 captured (Right hand, gesture: mafia)
✅ Sample 14/500 captured (Right hand, gesture: mafia)
✅ Sample 15/500 captured (Right hand, gesture: mafia)
✅ Sample 16/500 captured (Right hand, gesture: mafia)
✅ Sample 17/500 captured (Right hand, gesture: mafia)
✅ Sample 18/500 captured (Right hand